# Grow rich using Agentic AI for Personal Finance
### Using OpenAI Agents and Function_tool
### I haven't come across such simple way to define tools, all have easy way to invoke them; OpenAI allows, instead of calling a hosted function tool, you can write your tool right into the code
##### Thanks to Olivier Khatib, FRSA for this article https://www.linkedin.com/pulse/build-ai-personal-finance-agent-using-python-openais-khatib-frsa-mkg8f/

In [ ]:
import os
import sys
from openai import OpenAI
import logging

# Configure logging for detailed observability
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s - [%(filename)s:%(lineno)d]",
    filename="finance_agent_logs.log",
    filemode="w"  # Overwrite log file each run
)
logger = logging.getLogger(__name__)

# Add console output for real-time feedback
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(logging.Formatter("%(levelname)s: %(message)s"))
logger.addHandler(console_handler)

# Initialize OpenAI client
try:
    os.environ["OPENAI_API_KEY"] = ""  # Replace with your actual key
    client = OpenAI()
    logger.info("OpenAI client initialized successfully.")
except Exception as e:
    logger.error(f"Failed to initialize OpenAI client: {e}")
    sys.exit(1)

INFO: OpenAI client initialized successfully.


### Write the tools details using function_tool

In [2]:
from agents import function_tool
import json
from datetime import datetime

# Tool 1: Log an expense
@function_tool
def log_expense(category: str, amount: float) -> str:
    """Log an expense to a JSON file with timestamp and validation."""
    if not isinstance(amount, (int, float)) or amount < 0:
        logger.error(f"Invalid amount provided: {amount}")
        return "Error: Amount must be a positive number."
    if not category or not category.strip():
        logger.error("Empty category provided.")
        return "Error: Category cannot be empty."

    try:
        expense = {
            "category": category.strip(),
            "amount": float(amount),
            "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        with open("expenses.json", "a") as f:
            json.dump(expense, f)
            f.write("\n")
        logger.info(f"Expense logged: {expense}")
        return f"Logged expense: ${amount:.2f} in {category} on {expense['date']}."
    except Exception as e:
        logger.error(f"Failed to log expense: {e}")
        return "Error: Could not log expense. Please try again."

# Tool 2: Check budget adherence
@function_tool
def check_budget_adherence(monthly_income: float, total_expenses: float) -> str:
    """Evaluate expenses against the 50/30/20 budget rule."""
    if not isinstance(monthly_income, (int, float)) or monthly_income <= 0:
        logger.error(f"Invalid income: {monthly_income}")
        return "Error: Monthly income must be a positive number."
    if not isinstance(total_expenses, (int, float)) or total_expenses < 0:
        logger.error(f"Invalid expenses: {total_expenses}")
        return "Error: Total expenses must be non-negative."
    print (f"Monthly Income: {monthly_income}, Total Expenses: {total_expenses}")
    
    try:
        needs_limit = monthly_income * 0.50  # 50% for needs
        wants_limit = monthly_income * 0.30  # 30% for wants
        savings_limit = monthly_income * 0.20  # 20% for savings

        if total_expenses > needs_limit:
            logger.info(f"Budget check: Expenses (${total_expenses}) exceed 50% of income (${monthly_income})")
            return (f"Warning: Your expenses (${total_expenses:.2f}) exceed 50% of your income "
                    f"(${monthly_income:.2f}). Aim to keep needs below ${needs_limit:.2f}.")
        logger.info(f"Budget check: Expenses (${total_expenses}) within limit for income (${monthly_income})")
        return (f"Good news: Your expenses (${total_expenses:.2f}) are within 50% of your income "
                f"(${monthly_income:.2f}), leaving room for wants (${wants_limit:.2f}) and savings (${savings_limit:.2f}).")
    except Exception as e:
        logger.error(f"Budget check failed: {e}")
        return "Error: Could not evaluate budget."

### Setup Vector store for RAG budget guidance 
##### didn't work properly, need to check

In [3]:
def setup_vector_store():
    """Upload a budget guide PDF and create a vector store for semantic search."""
    try:
        # Verify file existence
        if not os.path.exists("budget_guide.pdf"):
            raise FileNotFoundError("budget_guide.pdf not found in current directory.")

        # Upload the file
        budget_file = client.files.create(
            file=open("budget_guide.pdf", "rb"),
            purpose="assistants"
        )
        logger.info(f"Uploaded budget guide with ID: {budget_file.id}")

        # Create a vector store
        vector_store = client.vector_stores.create(
            name="Budget Guide",
            file_ids=[budget_file.id],
            metadata={"description": "Personal finance budgeting tips"}
        )
        logger.info(f"Vector store created with ID: {vector_store.id}")
        return vector_store.id
    except FileNotFoundError as e:
        logger.error(f"File error: {e}")
        return None
    except Exception as e:
        logger.error(f"Failed to set up vector store: {e}")
        return None

## Define the agents, 4 agents being defined here , Budgeting Agent to help with budget decisions, Expense Tracker which will log your daily expenses , Investment Researcher for ideas on investments and last but not least is Triage agent for triaging 

In [4]:
from agents import Agent, WebSearchTool
from typing import Optional

def create_agents(vector_store_id: Optional[str]):
    """Define and configure agents for the personal finance assistant."""
    
    # Agent 1: Budgeting Assistant
    budget_agent = Agent(
        name="BudgetAgent",
        instructions="""
        You are a budgeting expert with access to a budget guide. Assist users by:
        1. Using file_search to retrieve tips from the budget guide (if available).
        2. If income and expenses are provided, use check_budget_adherence to evaluate against the 50/30/20 rule.
        3. If details are missing, ask specific questions (e.g., "What’s your monthly income?" or "What are your total expenses?").
        4. If file_search is unavailable, inform the user and proceed with budget analysis if possible.
        Provide clear, actionable advice with numbers where applicable.
        """,
        tools=[
        
            check_budget_adherence
        ],
        
    )

    # Agent 2: Expense Tracker
    expense_agent = Agent(
        name="ExpenseAgent",
        instructions="""
        You are an expense tracking specialist. Help users by:
        1. Using log_expense to record spending when given a category and amount.
        2. If details are missing, prompt the user (e.g., "What’s the amount?" or "Which category?").
        3. Confirm the expense with date and time after logging.
        4. If multiple expenses are mentioned, ask to process one at a time.
        Keep responses concise and confirm each action.
        """,
        tools=[log_expense],
       
    )

    # Agent 3: Investment Researcher
    investment_agent = Agent(
        name="InvestmentAgent",
        instructions="""
        You are an investment research expert. Assist users by:
        1. Using web_search_preview to find current investment options, trends, or stock data.
        2. Summarizing findings in 2-3 sentences with source citations (e.g., URLs).
        3. Tailor responses to specifics (e.g., price range, risk level) if provided; otherwise, give general advice.
        Ideal for queries like "best stocks under $100" or "should I invest in tech?".
        """,
        tools=[WebSearchTool()],
       
    )

    # Agent 4: Triage Agent
    triage_agent = Agent(
        name="TriageAgent",
        instructions="""
        You are the initial contact for finance queries. Analyze the input and route it to:
        - BudgetAgent for budgeting or spending analysis (e.g., "am I overspending?", "help me budget").
        - ExpenseAgent for tracking spending (e.g., "log $50 on food", "record an expense").
        - InvestmentAgent for investment questions (e.g., "best stocks under $100", "investment options").
        If the intent is unclear, ask for clarification with examples (e.g., "Do you want to log an expense, check your budget, or research investments?").
        Do not answer directly; delegate to the appropriate agent.
        """,
        handoffs=[budget_agent, expense_agent, investment_agent],
       
    )

    logger.info("All finance agents initialized successfully.")
    return triage_agent


### Create vector store, agent instances, specify queries you want and run the agents in sync mode
##### Queries could be from a chatbot and response could go back to a chatbit, here we are just showcasing the possibilies of agents

In [5]:
from agents import Runner
import nest_asyncio


def main():
    """Execute the personal finance assistant system with test queries."""
    # Step 1: Set up vector store
    vector_store_id = setup_vector_store()
    if not vector_store_id:
        logger.warning("Proceeding without budget guide access. BudgetAgent will rely on budget analysis only.")

    # Step 2: Create agents
    try:
        triage_agent = create_agents(vector_store_id)
    except Exception as e:
        logger.error(f"Failed to create agents: {e}")
        sys.exit(1)

    # Step 3: Define diverse test queries
    queries = [
        "Am I overspending if my income is $4000 and expenses are $2200?",
        "Log $50 spent on groceries today",
        "What are the best stocks under $100 to invest in right now?",
        "Help me with my finances"  # Vague query to test clarification
    ]

    # Step 4: Process queries with detailed execution
    for query in queries:
        logger.info(f"Starting query processing: '{query}'")
        try:
            output = Runner.run_sync(
                starting_agent=triage_agent,
                input=query,
                
              #  max_steps=5,  # Prevent infinite loops
               # max_time=30  # Timeout after 30 seconds
            )
            logger.info(f"Completed query '{query}' with response: {output}")
            print(f"\nQuery: {query}\nResponse: {output}")
        except Exception as e:
            logger.error(f"Error processing query '{query}': {e}")
            print(f"\nQuery: {query}\nError: Unable to process request. Check logs for details.")

if __name__ == "__main__":
    nest_asyncio.apply()
    main()

INFO: Uploaded budget guide with ID: file-3hiBWerWDL9KiiN4e6Gzup
INFO: Vector store created with ID: vs_680ccad1dfd08191bd359e6ebf06efaf
INFO: All finance agents initialized successfully.
INFO: Starting query processing: 'Am I overspending if my income is $4000 and expenses are $2200?'
Monthly Income: 4000.0, Total Expenses: 2200.0
INFO: Budget check: Expenses ($2200.0) exceed 50% of income ($4000.0)
INFO: Completed query 'Am I overspending if my income is $4000 and expenses are $2200?' with response: RunResult:
- Last agent: Agent(name="BudgetAgent", ...)
- Final output (str):
    Based on the 50/30/20 budget rule:
    
    1. **Needs (50%)**: These should be $2000 or less. At $2200, your expenses exceed this guideline.
       
    2. **Wants (30%)**: Ideally, this should be $1200 or less.
       
    3. **Savings (20%)**: This should be at least $800.
    
    ### Advice:
    - Consider reducing your expenses by $200 to meet the guideline.
    - Review which expenses are essential an